<a href="https://colab.research.google.com/github/changsin/FSDL/blob/main/notebooks/UFPR_License_Plate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brazilean license plate dataset

The dataset came from: https://web.inf.ufpr.br/vri/databases/ufpr-alpr/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
DATA_ROOT = "./drive/MyDrive/data/UFPR_YOLO"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8MB 5.2MB/s 
     |████████████████████████████████| 133kB 29.9MB/s 
     |████████████████████████████████| 174kB 39.1MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load the data

## Methods

In [4]:
IMAGE_SIZE = 224

In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import os
import glob

In [6]:
def load_images(path):
  path = os.path.join(path,'*')
  files = glob.glob(path)
  # We sort the images in alphabetical order to match them
  #  to the annotation files
  files.sort()

  X_raw = []
  for f1 in files:
    image = cv2.imread(f1)
    image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
    X_raw.append(np.array(image))

  return X_raw

def load_labels(path):
  path = os.path.join(path,'*')
  files = glob.glob(path)
  files.sort()

  y_raw = []
  for file in files:
      y_raw.append(extract_annotations(file, 0))
  return y_raw

In [7]:
def extract_annotations(label_file, class_id):
  labels = []
  with open(label_file, "r") as file:
    count = 0
    for line in file:
      tokens = [float(token) for token in line.split()]
      if tokens[0] == class_id:
        count += 1
        # print(line)
        labels.append(np.array(tokens[1:]))

    if count > 1:
      print("WARNING: More than one license plate was found: ", count, label_file)
    elif count == 0:
      print("WARNING: No license plate was found: ", count, label_file)

  return np.array(labels)

In [8]:
# transform to arrays and normalize
def normalize(X_raw, y_raw):
  X = np.array(X_raw)
  y = np.array(y_raw)
  y = y.reshape((y.shape[0], -1))

  #  Renormalisation
  X = X / IMAGE_SIZE
  y = y / IMAGE_SIZE

  return X, y

In [9]:
import matplotlib.patches as patches

def create_patch_rectangle(y, color):
  # width = y[2] - y[0]
  # height = y[3] - y[1]
  # return patches.Rectangle((y[0], y[1]),
  #                           width, height,
  #                           edgecolor=color, fill=False)
  # # in yolov5
  width = int(y[2])
  height = int(y[3])
  return patches.Rectangle((int(y[0] - width/2), int(y[1] - height/2)),
                           width, height,
                           edgecolor=color, fill=False)
def plot_images(X, y, limit=10):
  fig = plt.figure(figsize=(20,40))

  # The number of images for plotting is limited to 50
  end_id = len(y) if len(y) < limit else limit

  for i in range(0, end_id):
    axis = fig.add_subplot(10, 5, i+1)
    plt.axis('off')
    image = X[i]

    rect_ori = create_patch_rectangle(y[i]*IMAGE_SIZE, (0, 255/255, 0))
    axis.add_patch(rect_ori)
    plt.imshow(np.clip(image, 0, 1))
# plot_images(X_train_d[0], y_train_d[0])

## Execution

In [10]:
X_train_raw = load_images(DATA_ROOT + "/images/train/")
# X_test_raw = load_images(DATA_ROOT + "/images/test/")
# X_val_raw = load_images(DATA_ROOT + "/images/validation/")

y_train_raw = load_labels(DATA_ROOT + "/labels/train/")
# y_val_raw = load_labels(DATA_ROOT + "/labels/validation/")
# y_test_raw = load_labels(DATA_ROOT + "/labels/test/")


# # from sklearn.model_selection import train_test_split
# # TODO
# X_train_raw = X_raw

X_train, y_train = normalize(X_train_raw, y_train_raw)
# X_val, y_val = normalize(X_val_raw, y_val_raw)
# X_test, y_test = normalize(X_test_raw, y_test_raw)

# Clustering

### Methods

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn import preprocessing  # to normalise existing X

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
dist = tfp.distributions

# from keras.preprocessing import image
# from keras.applications.vgg16 import preprocess_input, decode_predictions

#Calculate similar matrics
def cosine_similarity(ratings):
    sim = ratings.dot(ratings.T)
    if not isinstance(sim,np.ndarray):
        sim = sim.toarray()
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim/norms/norms.T)

def get_feature_maps(input):
    # #Convert to VGG input format
    # NB: This messes up the existing data so skipping it
    #   similarity measures do not seem to be affected by this.
    # vgg_input = preprocess_input(input)

    #include_top=False == not getting VGG16 last 3 layers
    model = VGG16(weights = "imagenet", include_top=False)

    #Get features
    # feature_maps = model.predict(vgg_input)
    feature_maps = model.predict(input)

    return feature_maps, model

    # #Calculate similar metrics
    # features_compress = features.reshape(len(y_test), 7*7*512)
    # sim = cosine_similarity(features_compress)

# model_vgg16, feature_maps = get_feature_maps(X)

In [12]:
def find_clusters(X_np, K):
  fm_x, _ = get_feature_maps(X_np)
  # use cosine distance to find similarities
  fm_x_normalized = preprocessing.normalize(fm_x.reshape(len(fm_x), -1))

  clusters_x = KMeans(n_clusters=K, random_state=0).fit(fm_x_normalized)
  histo_x, bins = np.histogram(clusters_x.labels_, bins=range(K + 1))

  # plt.hist(bins[:-1], bins, weights=histo_x, histtype='step', label='x')
  plt.bar(bins[:-1], histo_x, align='center')

  return clusters_x

In [13]:
def to_cluster_ids(bins, labels):
  cluster_dict = dict()
  for cluster_id in bins:
    cluster_dict[cluster_id] = np.where(labels == cluster_id)[0]
  return cluster_dict

In [14]:
def to_clusters_dict(X, y, K):
  X_clusters = find_clusters(X, K)
  X_cluster_ids = to_cluster_ids(range(K), X_clusters.labels_)

  X_dict = {}
  y_dict = {}
  for id in range(K):
    ids = X_cluster_ids[id]
    X_dict[id] = X[ids]
    y_dict[id] = y[ids]

  return X_dict, y_dict

# merge all clusters to return the data
def get_merged_data(clusters_d):
  merged = []
  for id, data in clusters_d.items():
    merged = data if id == 0 else np.vstack((merged, data))
  return merged

In [15]:
from sklearn.model_selection import train_test_split

def partition_on_clusters(X_d, y_d, bins, val_size=0.1, test_size=0.2):
  X_train_d = dict()
  y_train_d = dict()
  X_val_d = dict()
  y_val_d = dict()
  X_test_d = dict()
  y_test_d = dict()

  # for each cluster reserve test_size portion for test data
  for id in bins:
    Xt_train, Xt_test, yt_train, yt_test = \
      train_test_split(X_d[id], y_d[id], test_size=0.2, shuffle=False)
    Xt_train, Xt_val, yt_train, yt_val = \
      train_test_split(Xt_train, yt_train, test_size=0.1, shuffle=False)

    X_train_d[id] = Xt_train
    y_train_d[id] = yt_train

    X_val_d[id] = Xt_val
    y_val_d[id] = yt_val

    X_test_d[id] = Xt_test
    y_test_d[id] = yt_test

  return X_train_d, y_train_d, \
         X_val_d, y_val_d, \
         X_test_d, y_test_d

### Execution

In [ ]:
K = 5
# bins, cluster_x_counts = np.unique(clusters_x.labels_, return_counts=True)
X_train_d, y_train_d = to_clusters_dict(X_train, y_train, K)
# X_val_d, y_val_d = to_clusters_dict(X_val, y_val, K)
# X_test_d, y_test_d = to_clusters_dict(X_test, y_test, K)

In [ ]:
X_train_d, y_train_d, X_val_d, y_val_d, X_test_d, y_test_d = \
          partition_on_clusters(X_train_d, y_train_d, range(K))

# Models

## Methods

In [ ]:
def create_model(train_size, probability=True):
  kl_divergence_fn = lambda q, p, _: dist.kl_divergence(q, p) / tf.cast(y_train.shape[0], dtype=tf.float32)

  model = Sequential()
  model.add(VGG16(weights="imagenet", include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
  model.add(Flatten())
  model.add(Dense(128, activation="relu"))
  model.add(Dense(128, activation="relu"))
  model.add(Dense(64, activation="relu"))

  if probability:
    model.add(tfp.layers.DenseFlipout(4, activation="sigmoid", kernel_divergence_fn=kl_divergence_fn))
  else:
    model.add(Dense(4, activation="sigmoid"))

  model.layers[-6].trainable = False
  model.summary()

  model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

  return model

# Train

## Methods

In [ ]:
def train(model,
          X_train, y_train,
          X_val, y_val,
          X_test, y_test,
          epochs=50, batch_size=16,
          is_plot_predictions=False):
  train_history = model.fit(x=X_train, y=y_train,
                            validation_data=(X_val, y_val),
                            epochs=epochs, batch_size=batch_size, verbose=1,
                            callbacks=[wandb.keras.WandbCallback(data_type="image",
                            save_model=False)])
  # Test
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("Score : %.2f%%" % (scores[1]*100))

  test_loss, test_accuracy = model.evaluate(X_test, y_test, steps=int(100))

  print("Test results \n Loss:",test_loss,'\n Accuracy',test_accuracy)

  y_preds = sample_predictions(model, X_test, iterations=1)
  # y_preds = model.predict(X_test)

  # # TODO:
  # # Hack to fix erroneous predictions
  # y_preds = fix_predictions(y_preds)
  if is_plot_predictions:
    plot_predictions(X_test, y_test, y_preds)

  # averaged_predictions = average_sample_preds(y_preds)
  # y_test = np.array([to_rect(y*IMAGE_SIZE) for y in y_test])
  # rectified_predictions = np.array([to_rect(y*IMAGE_SIZE) for y in averaged_predictions])

  # # print(rectified_predictions)
  # m_ap = calculate_map(y_test*IMAGE_SIZE, rectified_predictions*IMAGE_SIZE)
  return model

### Sample predictions

In [ ]:
# run predictions many times to get the distributions
def sample_predictions(model, samples, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(samples).numpy())

    predicted = np.array(predicted)
    # predicted = np.concatenate(predicted, axis=1)

    # predicted = np.array([model_prob.predict(np.expand_dims(X_test[1], [0])) for i in range(iterations)])
    # predicted = np.concatenate(predicted, axis=1)
    reshaped = np.array([predicted[:, column] for column in range(0, predicted.shape[1])])

    return reshaped

def predict_on_cluster(model, X_test, y_test, is_plot_predictions=False, iterations=50):
  test_accuracy = 0
  test_loss, test_accuracy = model.evaluate(X_test, y_test, steps=1)
  y_preds = sample_predictions(model, X_test, iterations=iterations)

  # TODO:
  # Hack to fix erroneous predictions
  # y_preds_fixed = fix_predictions(y_preds)
  if is_plot_predictions:
    plot_predictions(X_test, y_test, y_preds)

  preds_avg = average_sample_preds(y_preds)
  rectified_y_test = np.array([to_rect(y*IMAGE_SIZE) for y in y_test])
  rectified_predictions = np.array([to_rect(y*IMAGE_SIZE) for y in preds_avg])

  m_ap = calculate_map(rectified_y_test*IMAGE_SIZE, rectified_predictions*IMAGE_SIZE)
  stds = np.mean(np.std(y_preds, axis=1), axis=1)

  return y_preds, m_ap, np.mean(stds, axis=0), test_accuracy

In [ ]:
def predict_on_models(X, y, bins, models):
  stats = []
  for model in models:
    cluster_stats = []
    for clst_id in bins:
      # y_preds, m_ap, accuracy, mstd = predict_on_cluster(model, X[clst_ids[clst_id]], y[clst_ids[clst_id]])
      y_preds, m_ap, std, accuracy = predict_on_cluster(model, X[clst_id], y[clst_id])
      print("{} mAP: {:0.2f} std: {:0.2f} acc: {:0.2f}".format(clst_id,
                                                               m_ap['avg_prec'],
                                                               std,
                                                               accuracy))
      cluster_stats.append([np.round(m_ap['avg_prec'], 3), np.round(std, 3), np.round(accuracy, 3)])

    stats.append(cluster_stats)

  return np.array(stats)

## Execution

In [ ]:
X_train = get_merged_data(X_train_d)
y_train = get_merged_data(y_train_d)

X_val = get_merged_data(X_val_d)
y_val = get_merged_data(y_val_d)

X_test = get_merged_data(X_test_d)
y_test = get_merged_data(y_test_d)

In [ ]:
wandb.init(project="UFPR-cnn",
           config={
               "batch_size": 16,
               "learning_rate": 0.01,
               "dataset": "UFPR-cnn",
           })

model_cnn = create_model(y_train.shape[0], probability=False)
model_cnn = train(model_cnn,
                  X_train, y_train,
                  X_val, y_val,
                  X_test, y_test, epochs=100)

In [ ]:
wandb.init(project="UFPR-prob",
           config={
               "batch_size": 16,
               "learning_rate": 0.01,
               "dataset": "UFPR-prob",
           })

model_prob = create_model(y_train.shape[0], probability=True)
model_prob = train(model_prob,
                   X_train, y_train,
                   X_val, y_val,
                   X_test, y_test)

# Predict

## Methods

### Plot predictions

In [ ]:
def plot_predictions(X, y_gt, y_preds):
  fig = plt.figure(figsize=(20,40))

  # The number of images for plotting is limited to 50
  end_id = len(y_gt) if len(y_gt) < 50 else 50

  y_preds_avg = average_sample_preds(y_preds)
  stds = np.std(y_preds, axis=1)
  mean_stds = np.mean(stds, axis=1)

  for i in range(0, end_id):
    axis = fig.add_subplot(10, 5, i+1)
    plt.axis('off')
    image = X[i]

    rect_ori = create_patch_rectangle(y_gt[i]*IMAGE_SIZE, (0, 255/255, 0))
    axis.add_patch(rect_ori)

    # for each test image, there could be multiple predictions
    for y_pred in y_preds[i]:
      rect_pred = create_patch_rectangle(y_pred*IMAGE_SIZE, (255/255, 0, 0))
      axis.add_patch(rect_pred)

    iou = bb_iou(to_rect(y_preds_avg[i]*IMAGE_SIZE), to_rect(y_gt[i]*IMAGE_SIZE))
    plt.title("IOU: {:0.2f} std: {:0.2f}".format(iou, mean_stds[i]))
    # plt.title("mean std: {:0.2f}".format(mean_stds[sample_ids[i]]))
    plt.imshow(np.clip(image, 0, 1))

In [ ]:
def plot_stats(stats, types, titles):
  x_bar = np.arange(K)
  bar_width = .35

  fig = plt.figure(figsize=(20,40))

  for i, stat in zip(range(len(stats)), stats):
    ax = fig.add_subplot(10, 4, i+1)
    # plt.axis('off')

    for t in types:
      if "mAP" == t:
        rects1 = ax.bar(x_bar - bar_width/3, stat[:, 0], label="mAP")
      if "std" == t:
        rects2 = ax.bar(x_bar + bar_width/3, stat[:, 1], label="std")
      if "accuracy" == t:
        rects3 = ax.bar(x_bar + bar_width/3, stat[:, 2], label="accuracy")

    ax.set_xticks(x_bar)
    ax.set_xticklabels(x_bar)
    ax.set_title(titles[i])

    ax.legend()

  fig.tight_layout()

In [ ]:
# plot line graph for comparing different noise level results
def plot_stats_by_type(stats, x_bins, column, title, clusters=bins, merge=False):
  fig = plt.figure(figsize=(20,40))
  ax = fig.add_subplot(10, 4, 1)

  if merge:
    p = ax.plot(x_bins, [np.mean(run) for run in stats_noise[:, :, 1]], label='avg')
  else:
    x_bins_len = len(x_bins)
    for cluster_id in range(len(clusters)):
      p = ax.plot(x_bins, stats[:x_bins_len, :, column][:, cluster_id], label=cluster_id)
    # p = ax.plot(noise_levels, stats[:bins, :, column][:, 0], color='green', label='0')

  ax.set_title(title)
  ax.legend()

## Execution

In [ ]:

_, m_ap, std, accuracy = predict_on_cluster(model_prob, X_test_d[0], y_test_d[0], is_plot_predictions=True, iterations=100)
print("{} mAP: {:0.2f} std: {:0.2f} acc: {:0.2f}".format(0,
                                                          m_ap['avg_prec'],
                                                          std,
                                                          accuracy))
